In [1]:
import pandas as pd
import numpy as np
import random
from datetime import date, datetime

In [2]:
products = pd.read_csv('raw-product.csv')
products.head()

,ProID,CatID,Seller,ProName,StartPrice,StepPrice,SellPrice,UploadDate,EndDate,AutoExtend,ProState,AllowAllUsers,CurrentWinner
0,1,1,thanhhoang4869@gmail.com,Silver Ring STYLE By PNJ 0000H000007,250000,50000,2000000,2021-12-07,2022-01-31,0,1,0,vminathanhv@gmail.com
1,2,1,thanhhoang4869@gmail.com,Ruby Ring STYLE By PNJ XM00X000009,250000,50000,2000000,2021-12-07,2022-01-31,0,1,0,springfieldcaptain@gmail.com
2,3,1,kysutainangqsb@gmail.com,Gold Ring with Ruby STYLE By PNJ XM00C060000,250000,50000,2000000,2021-12-07,2022-01-31,0,1,0,vminathanhv@gmail.com
3,4,1,kysutainangqsb@gmail.com,Diamond Ring STYLE By PNJ DNA XMXMX000004,500000,100000,2000000,2021-12-07,2022-01-31,0,1,0,NaN
4,5,1,kysutainangqsb@gmail.com,10K White Gold Ring PNJ XMXMW001741,500000,100000,2000000,2021-12-07,2022-01-31,0,1,0,NaN


In [3]:
accounts = pd.read_csv('raw-account.csv')
emails = accounts['Email']
emails.shape

(8,)

In [4]:
def get_date_datetime(dt):
    return datetime.strptime(dt,'%Y-%m-%d %H:%M:%S')
def get_date_date(date):
    return get_date_datetime(("%s 00:00:00")%(str(date)))

In [5]:
biddings = pd.DataFrame(columns=["ProID","Bidder","Time","Price","MaxPrice"])
existed_bidding = [1,2,3,7,8,30,33]
# existed_bidding = []
MIN_BID = 5
MAX_BID = 10
count = 0

log_bidding = open('log_bidding.txt','w')
winners = []

for index, product in products.iterrows():
    if (product['ProID'] in existed_bidding):
        continue   
    nBidding = random.randint(MIN_BID,MAX_BID)
    proID = product['ProID']
    seller = product["Seller"]
    startPrice = product['StartPrice']
    stepPrice = product['StepPrice']
    sellPrice = product['SellPrice']
    uploadDate = get_date_date(product['UploadDate']).timestamp()*1000
#     endDate = get_date_date(product['EndDate']).timestamp()*1000
    endDate = datetime.now().timestamp()*1000
    currentWinner = None
    currentPrice = startPrice
    currentMaxPrice = startPrice

    deltaMillis = (endDate-uploadDate)//(nBidding+2)

    stringBuilder = ""
    stringBuilder += "Running product: {}\n".format(proID)
    stringBuilder += "Enumeration:... {} {} {} {} {} {}\n".format(
        seller,
        startPrice,
        stepPrice,
        sellPrice,
        datetime.fromtimestamp(uploadDate/1000),
        datetime.fromtimestamp(endDate/1000),
        '\n'
    )

    for enum in range(nBidding):
        bidder = None
        while (bidder==None or bidder==seller or bidder==currentWinner):
            bidder = random.choice(emails)

        minStep = 1
        maxStep = (sellPrice-currentPrice)//stepPrice-1
        if (minStep>maxStep):
            break

        bidMaxPrice = currentPrice+random.randint(minStep,maxStep-(nBidding-enum))*stepPrice
        bidPrice = None
        if (bidMaxPrice<=currentMaxPrice):
            bidMaxPrice = currentMaxPrice
            bidPrice = currentPrice+stepPrice
            bidder = currentWinner
        else:
            bidPrice = currentMaxPrice+stepPrice
            currentMaxPrice = bidMaxPrice
            currentWinner = bidder
        currentPrice = bidPrice

        bidMillis = uploadDate+deltaMillis*(enum+1)
        bidDateTime = datetime.fromtimestamp(bidMillis/1000)
        bidFormat = bidDateTime.strftime("%Y-%m-%d %H:%M:%S")
        
        data = [proID,bidder,bidFormat,bidPrice,bidMaxPrice]
        stringBuilder += "{} {} {}".format(enum+1,data,'\n')
        biddings.loc[count]=data
        count+=1
    stringBuilder+="\n"
    log_bidding.write(stringBuilder)
    products.at[index,'CurrentWinner']=currentWinner
log_bidding.close()
print("Done generate")

Done generate


In [6]:
print(biddings.describe())

        ProID                 Bidder                 Time   Price  MaxPrice
count     269                    269                  269     269       269
unique     38                      8                   46      46        38
top        26  sinsin.4869@gmail.com  2021-12-24 22:09:27  600000   1200000
freq       10                     44                   15      19        16


In [7]:
print(products.head())

   ProID  CatID                    Seller  \
0      1      1  thanhhoang4869@gmail.com   
1      2      1  thanhhoang4869@gmail.com   
2      3      1  kysutainangqsb@gmail.com   
3      4      1  kysutainangqsb@gmail.com   
4      5      1  kysutainangqsb@gmail.com   

                                        ProName  StartPrice  StepPrice  \
0          Silver Ring STYLE By PNJ 0000H000007      250000      50000   
1            Ruby Ring STYLE By PNJ XM00X000009      250000      50000   
2  Gold Ring with Ruby STYLE By PNJ XM00C060000      250000      50000   
3     Diamond Ring STYLE By PNJ DNA XMXMX000004      500000     100000   
4           10K White Gold Ring PNJ XMXMW001741      500000     100000   

   SellPrice  UploadDate     EndDate  AutoExtend  ProState  AllowAllUsers  \
0    2000000  2021-12-07  2022-01-31           0         1              0   
1    2000000  2021-12-07  2022-01-31           0         1              0   
2    2000000  2021-12-07  2022-01-31           0     

In [8]:
biddings.to_csv('final-bidding.csv',index=False)
products.to_csv('final-product.csv',index=False)

In [9]:
final_sql = open('file-sql.sql','w')

# ============ Bidding ===========
sql = []
prevID = None
for index,row in biddings.iterrows():
    s = "INSERT INTO `bidding` VALUES ({},'{}','{}',{},{});\n".format(
        row['ProID'],
        row['Bidder'],
        row['Time'],
        row['Price'],
        row['MaxPrice']
    )
    if (prevID==None):
        prevID=row['ProID']
    elif (row['ProID']!=prevID):
        sql.append('\n')
        prevID=row['ProID']
    sql.append(s)
final_sql.writelines(sql)
final_sql.write("\n")

# ============= Product ===========
sql = []
prevID = None
for index,row in products.iterrows():
    s = "INSERT INTO `product` VALUES ({},{},'{}','{}',{},{},{},'{}','{}',{},{},{},'{}');\n".format(
        row['ProID'],
        row['CatID'],
        row['Seller'],
        row['ProName'],
        row['StartPrice'],
        row['StepPrice'],
        row['SellPrice'],
        row['UploadDate'],
        row['EndDate'],
        row['AutoExtend'],
        row['ProState'],
        row['AllowAllUsers'],
        row['CurrentWinner']
    )
    
    if (prevID==None):
        prevID=row['CatID']
    elif (row['CatID']!=prevID):
        sql.append('\n')
        prevID=row['CatID']
    sql.append(s)
final_sql.writelines(sql)

final_sql.close()